In [ ]:
"""
4get Instance Checker - Only Working if Returns ≥1 Result
Auto-fixes URLs · No proxy · Simple & Reliable
"""

import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

INPUT_FILE = "inpurl.txt"
OUTPUT_FILE = "working_urls.txt"
TIMEOUT = 15

# Simple test query that almost always returns results
TEST_QUERY = "github"

def fix_url(url):
    url = url.strip()
    if not url or url.startswith("#"):
        return None
    if not url.startswith(("http://", "https://")):
        url = "https://" + url
    return url.rstrip("/")

def is_working(url):
    if not url:
        return None
    api_url = f"{url}/api/v1/web"
    try:
        r = requests.get(
            api_url,
            params={"s": TEST_QUERY, "scraper": "startpage"},
            headers={"User-Agent": "Mozilla/5.0"},
            timeout=TIMEOUT,
            verify=False
        )
        if r.status_code != 200:
            print(f"× {url} → HTTP {r.status_code}")
            return None

        data = r.json()
        results = data.get("web", [])
        
        if data.get("status") == "ok" and len(results) >= 1:
            print(f"✓ {url} → {len(results)} results")
            return url
        else:
            print(f"× {url} → No results (empty response)")
            return None
    except Exception as e:
        print(f"× {url} → Error: {e}")
        return None

# === MAIN ===
print("Checking 4get instances (must return ≥1 result)...\n")

urls = []
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        fixed = fix_url(line)
        if fixed:
            urls.append(fixed)

if not urls:
    print("No valid URLs found in input file.")
    exit()

print(f"Testing {len(urls)} instances...\n")

working = []
with ThreadPoolExecutor(max_workers=25) as executor:
    futures = [executor.submit(is_working, url) for url in urls]
    for future in as_completed(futures):
        result = future.result()
        if result:
            working.append(result)

working = sorted(working)

output  = '["' + '","'.join(working) + '"]' if working else "[]"

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(output + "\n")

print("\n" + "="*60)
print(f"Done! {len(working)} truly working instance(s) found:")
print(output)
print(f"\nSaved to → {OUTPUT_FILE}")
print("="*60)